# Bi-criteria SUE
Cantarella, G. E., & Binetti, M. (1998). Stochastic Equilibrium Traffic Assignment with Value‐of‐time Distributed Among Users. International Transactions in Operational Research, 5(6), 541-553.

In [1]:
using DelimitedFiles:readdlm

## Problem setting.

<img src="data/Caliper_net.png" width="800">

In [2]:
netFile = "data/network.csv"
csvdata = readdlm(netFile, ',', header=true)
data = csvdata[1]
header = csvdata[2]

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

In [3]:
display(header)
display(data)

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

7×10 Matrix{Float64}:
 1.0  2.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 1.0  4.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  3.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  5.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 3.0  6.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 4.0  5.0  50.0  20.0  10.0  0.15  4.0  60.0  4.0  1.0
 5.0  6.0  50.0  20.0  10.0  0.15  4.0  60.0  2.0  1.0

In [4]:
# BPR function
function BPR(cap, fft, alpha, beta,  flow)
    return fft * (1 + alpha * (flow/cap) ^ beta)
end

BPR (generic function with 1 method)

In [5]:
# Logit probability
function Logit(cost)
    theta = 1.0
    util = -cost
    return exp.(theta * util) / sum(exp.(theta * util))
end

Logit (generic function with 1 method)

In [6]:
links = []
for l in zip(round.(Int, data[:,1]), round.(Int, data[:,2]))
    push!(links, l)
end
cap = data[:, 3]
fft = data[:, 5] / 60
alpha = data[:, 6]
beta = data[:, 7]
toll = data[:, 9]
dem = 100.0

100.0

In [7]:
# link-path incidence matrix (a 7 x 3 matrix)
display(links)
Δ = Matrix([
    1 1 0;
    0 0 1;
    1 0 0;
    0 1 0;
    1 0 0;
    0 0 1;
    0 1 1
    ])

7-element Vector{Any}:
 (1, 2)
 (1, 4)
 (2, 3)
 (2, 5)
 (3, 6)
 (4, 5)
 (5, 6)

7×3 Matrix{Int64}:
 1  1  0
 0  0  1
 1  0  0
 0  1  0
 1  0  0
 0  0  1
 0  1  1

## Formulation
It is a fixed point problem with respect to path flow. 
$$
\mathbf{f} = F(\mathbf{f}) \Rightarrow \mathbf{f = q * Pr(-\Delta^{T}t(\Delta f))}
$$

Alternatively, we can also write it as a fixed point problem with respect to link flow.
$$
\mathbf{x} = G(\mathbf{x}) \Rightarrow \mathbf{x = \Delta q * Pr(-\Delta^{T}t(x))}
$$

We use MSA method to solve this fixed point problem with respect to path flow. 

## VOT is a deterministic variable
We can directly extend the model by inserting the generalized cost mapping.

In [8]:
# Deterministic VOT
vot = 10

10

In [9]:
# Use MSA to solve this problem. Time-based BiSUE model.
f = [100; 0; 0]
f_aux = []
for iter in 1:10000
    f_aux = dem * Logit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) + 1/vot * Δ' * toll)
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("g     = $(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) + 1/vot * Δ' * toll)")

f_aux = [35.85607790051056, 32.96585336041948, 31.178068739069964]
f     = [35.85706201081719, 32.96454234750634, 31.17839564167633]
g     = [1.205918462719906, 1.2899592741331283, 1.3457166300689232]


In [10]:
# Use MSA to solve this problem. Money-based BiSUE model.
f = [100; 0; 0]
f_aux = []
for iter in 1:10000
    f_aux = dem * Logit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) * vot + Δ' * toll)
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("g     = $(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) * vot + Δ' * toll)")

f_aux = [42.30235678066016, 30.347726442456047, 27.34991677688379]
f     = [42.302356899637395, 30.347726255412482, 27.349916844950236]
g     = [12.740975493883631, 13.07309669082669, 13.177104808268048]


In the case of SUE, even though taking the same parameters and using a linear combination to form the generalized cost, the time-based BiSUE and money-based BiSUE will still produce different results. Because the generalized costs in the two settings have different magnitudes and the Logit model uses the absolute difference to determine the probability. Therefore, it will produce different choice probabilities. This inconsistency will resolve if we take the relative difference to calculate the probability.

## VOT is a discrete random variable

In [11]:
# VOT is a discrete random variable
vot = [10; 20; 30] # the possible VOT value
η = [0.3; 0.5; 0.2] # probability of a user to have a VOT

3-element Vector{Float64}:
 0.3
 0.5
 0.2

In [12]:
# Use MSA to solve this problem. Time-based BiSUE model.
f = [100; 0; 0]
f_aux = []
for iter in 1:10000
    f_aux = [0.0; 0.0; 0.0]
    for (i, v) in enumerate(vot)
        f_aux += dem * η[i] * Logit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) + 1/vot[i] * Δ' * toll)
    end
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("g(1)  = $(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) + 1/vot[1] * Δ' * toll)")
println("g(2)  = $(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) + 1/vot[2] * Δ' * toll)")
println("g(3)  = $(Δ' * BPR.(cap, fft, alpha, beta, Δ * f) + 1/vot[3] * Δ' * toll)")

f_aux = [33.2338840673759, 32.3287185317741, 34.43739740085001]
f     = [33.235031032018426, 32.32721436934061, 34.43775459864037]
g(1)  = [1.1673315944823135, 1.2693602444698213, 1.3630240531801023]
g(2)  = [1.1673315944823135, 1.1693602444698215, 1.063024053180102]
g(3)  = [1.1673315944823135, 1.136026911136488, 0.9630240531801022]


## VOT is a continuous random variable

In [13]:
# Let's consider a uniformly distributed VOT with support [10, 30]
function PDF(x::Float64)
    if x < 10.0 || x > 30.0
        return 0.0
    else
        return 1.0 / (30 - 10)
    end
end

PDF (generic function with 1 method)

## See the PDF file. It is hard to get the analytical for for this network.